[//]: <> (MARIMOUTOU Mourougen / LACHEZE Enzo - Polytech Clermont 2023-2024)

<u>Etape 1 - 4</u>

- [x] Capture et création de la base de données annotées + prétraitement
- [x] Data augmentations de la base de données
- [x] Entrainement du réseau de neurones
- [X] Test du modéle sur une image et sur un flux vidéo

Installation des librairie utilisé

In [1]:
%pip install opencv-python
%pip install scikit-learn
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\mouro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\mouro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\mouro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### I.1- Capture des Images et labellisation

On choisis arbitrairement les 10 signes qui seront reconnus par le programme.  
Puis on effectue un certains nombre de capture d'écran ou on récupère des images des mains réalisant les 10 différents signes, on prend 20 images par signes (10 pour chaque mains).  
Le module 'HandTrackingModule' d'opencv permet de traquer automatiquement les mains et de définir la fameuse zone d'intérêt (ROI) afin de bien capturer les images.

In [2]:
import os
import cv2
from cvzone import HandTrackingModule

KeyboardInterrupt: 

In [ ]:
## Declaration des variables globales
IMG_DIRECTORY = 'images/labelledimg/' 

label = input("Nom du label : ")
num_imgs=20                                     # nombre d'images prise par label
cnt_img=0 

detector = HandTrackingModule.HandDetector()

In [ ]:
## Initialisation de la webcam
capture = cv2.VideoCapture(0)

## Creation du repertoire pour les images annotés
os.makedirs(IMG_DIRECTORY+'/'+label, exist_ok=True)

In [ ]:
while True:
    
    ## Capture du flux de la webcam
    ret,frame = capture.read()
    img_copy = frame.copy()
    hands, frame = detector.findHands(frame)

    if not ret:
        print("Erreur lors de la lecture de l image")
        break

    ## Mirroir de l'image pour plus de naturel
    frame = cv2.flip(frame, 1)

    ## Affichage du flux
    cv2.imshow("Image", frame)

    key = cv2.waitKey(1)                        #lecture des entrées claviers
    # Quit
    if key%256 == 113:  #(q for quit)
        print("quit...")
        break
    # SPACE
    elif key%256 == 32:

        bbox_value = hands[0].get('bbox')

        ## Capture de la zone d'interet(ROI) et enregistrement
        roi = img_copy[bbox_value[1]:bbox_value[1] + bbox_value[3], bbox_value[0]:bbox_value[0] + bbox_value[2]]

        output_name = "{0}/{1}/{1}_{2}.png".format(IMG_DIRECTORY,label,cnt_img)
        cv2.imwrite(output_name, roi)
        print("out : {}".format(output_name))
        cnt_img += 1
        if cnt_img > num_imgs:
            print("finished !")
            break


capture.release()
cv2.destroyAllWindows()  

### I.2 - Pré-traitement des données

On prétraite les images prises afin de mieux les traiter par la suite. Pour cela, on applique un filtre Gaussien à toutes les images car ce filtre permet d'uniformiser les parties d'une image en les floutant et donc en harmonisant les détails de celle-ci.

In [ ]:
import cv2
import numpy as np
import os

In [ ]:
## Declaration des variables globales
SRC_IMG_DIRECTORY=''
OUT_IMG_DIRECTORY=''

## Creation du repertoire pour les images traiter
os.makedirs(OUT_IMG_DIRECTORY, exist_ok=True)

In [ ]:
## Fonction du filtre gaussien
def gauss_filter(image,filename):

    ## Chargement l'image
    img = cv2.imread(image)
    ## Filtrage de l'image
    filt_img=cv2.GaussianBlur(img,(5,5),0)
    ## Enregistrement
    output_name = "{0}/{1}".format(OUT_IMG_DIRECTORY,filename)
    cv2.imwrite(output_name, filt_img)

In [ ]:
## Traitement
for directory in os.listdir(SRC_IMG_DIRECTORY):
    directory_path = os.path.join(SRC_IMG_DIRECTORY, directory)
    for filename in os.listdir(directory_path):
        img = os.path.join(directory_path, filename)
        gauss_filter(img,filename)

### II - Data Augmentation

Augmentor est un logiciel utilisé pour l'augmentation d'image. Il réalise des opérations typiquement utilisé pour l'apprentissage automatique. Augmentor utilise plusieurs classes pour la modification d'images comme les classes 'Rotation' ou 'Recadrage'. Les classes d'Augmentor permettent de couvrir la plupart des besoins pour l'aumgentation de d'une banque de données d'images.
L'augmentation d'images étant souvent une procédure en plusieurs étapes, Augmentor utilise une approche basée sur un pipeline, où les opérations sont ajoutées séquentiellement afin de générer un pipeline. Les images passent ensuite par ce pipeline, où chaque opération est appliquée à l'image au fur et à mesure de son passage.
Chaque fonction d'Augmentor possède au moins un paramètre de probabilité,  qui contrôle la probabilité d'application de l'opération à chaque image vue lors de son passage dans le pipeline.

Installation de Augmentor pour réaliser la data augmentaiton

In [ ]:
%pip install Augmentor

In [ ]:
import Augmentor
import os


SRC_DIR = os.path.abspath('Code/images/images_final/')

# On choisit arbitrairement un nombre d'échantillons égal à 400
echantillon = 400

In [ ]:
# Vérifier si le répertoire source existe
if not os.path.exists(SRC_DIR):
    print(f"Le répertoire source '{SRC_DIR}' n'existe pas.")
else:
    # On place toutes les images du répertoire dans le pipeline d'Augmentor
    p = Augmentor.Pipeline(SRC_DIR)

    # Les opérations à effectuer dans le pipeline ainsi que les paramètres ont été choisit arbitrairement par nous mêmes
    p.flip_top_bottom(probability=0.8)
    p.random_brightness(probability=0.3, min_factor=0.3, max_factor=1.2)
    p.random_distortion(probability=0.6, grid_width=4, grid_height=4, magnitude=8)

    p.sample(echantillon)
    #Les images qui ont été modifiés pour l'augmentation de données sont placées dans le répertoire source

### III - Création d'un modéle et entrainement

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [ ]:
# Chargement des données images - labels
DATA_DIR = "final"
LABELS = sorted(os.listdir(DATA_DIR))
print(LABELS)
image = []
id = []
for ID, label in enumerate(LABELS):
    for file in os.listdir(DATA_DIR +"/"+label):
        filepath = DATA_DIR + "/" + label + "/" + file
        img = cv2.resize(cv2.imread(filepath),(100,100))
        image.append(img)
        id.append(ID)
image = np.asarray(image)
id = np.asarray(id)
print(image.shape, id.shape)

In [ ]:
# Decoupage du dataset en pourcentage
image_train, image_test, id_train, id_test = train_test_split(image, id, test_size = 0.7)


# Transformation des labels en vecteurs
id_train = to_categorical(id_train)
id_test = to_categorical(id_test)
print(id_train.shape, id_test.shape)

 # Normalisation des images
image_train = image_train / 255.0
image_test = image_test / 255.0

In [ ]:
# Création du modèle
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu', input_shape=(100,100,3)),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

# Compilation du modèle
# Definition d'un taux d'apprentissage faible pour un modele stable
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Entrainement du modèle
history = model.fit(image_train, id_train, epochs=40,validation_data=(image_test, id_test))

# Visualisation des résultats
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

# Plot Accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Évaluation sur le jeu de test
test_loss, test_acc = model.evaluate(image_test, id_test)
print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")

In [ ]:
# Sauvegarde du modèle
model.save('modeles/model.h5')

### IV - Test du modéle

In [ ]:
import cv2
import numpy as np
import os
from keras.models import load_model

#### IV-1 - Test sur une image

In [ ]:
# Chargement du modèle
model = load_model('modeles/model.h5')

# Déclaration des classe
classlabel ={
    0:"A",
    1:"amour",
    2:"C",
    3:"I",
    4:"L",
    5:"W",
    6:"y",
    7:"hello",
    8:"oui",
    9:"peace"
}

# Chargement de l'image
image = cv2.imread("val/val.png")


#Prétraitements identique à l'entrainement du modèle
image=cv2.GaussianBlur(image,(5,5),0)
image = cv2.resize(image,(100,100))
image = np.array(image)/ 255.0
image = np.expand_dims(image, axis=0)

# Predictions
predictions = model.predict(image)

# Recupere la classe avec le plus de votes
class_index=np.argmax(predictions[0])

# Affichage du résultat
print(classlabel[class_index])

#### IV-2 - Test sur un flux vidéo

Nous pouvons généraliser le script ci-dessus afin de traiter directement les images à partir d'un flux vidéo.

In [ ]:
## Initialisation de la webcam et du detecteur de main
capture = cv2.VideoCapture(0)
detector = HandTrackingModule.HandDetector()

# Chargement du modèle
model = load_model('modeles/model1801v2.h5')

# Déclaration des classes
classlabel ={
    0:"A",
    1:"C",
    2:"I",
    3:"L",
    4:"W",
    5:"Y",
    6:"amour",
    7:"hello",
    8:"oui",
    9:"peace"
}


while True:
    ## Capture du flux de la webcam
    ret,frame = capture.read()
    img_copy = frame.copy()
    hands, frame = detector.findHands(frame)

    if not ret:
        print("Erreur lors de la lecture de l image")
        break
    ## Mirroir de l'image pour plus de naturel
    frame = cv2.flip(frame, 1)

    key = cv2.waitKey(1)
    # Quit
    if key%256 == 113:
        print("quit...")
        break
    if hands != [] :

        bbox_value = hands[0].get('bbox')

        ## Capture de la zone d'interet(ROI)
        image = img_copy[bbox_value[1]:bbox_value[1] + bbox_value[3], bbox_value[0]:bbox_value[0] + bbox_value[2]]

        #Prétraitements identique aux images d'entraînement
        image=cv2.GaussianBlur(image,(5,5),0)
        image = cv2.resize(image,(100,100))
        image = np.array(image)/ 255.0
        image = np.expand_dims(image, axis=0)

        #predictions
        predictions = model.predict(image)
        class_index = np.argmax(predictions[0])
        #print(classlabel[class_index])
        
        #Affiche du texte
        cv2.putText(frame, f"{classlabel[class_index]}", (30, 30), cv2.FONT_HERSHEY_PLAIN,2,(0,255,15), 2)

    ## Affichage du flux
    cv2.imshow("Image", frame)

capture.release()
cv2.destroyAllWindows()